# KLP Notebook
This notebook can be either used as a reference to how to run the kelp coverage calculations or can be used directly to run the calculations. Results are saved site by site and saved in the dir. Test data can be found on github under the `tests/test_data` directory. Recommended to run on a GPU

# Import the package

In [3]:
!pip install kelp-coverage --quiet
from google.colab.patches import cv2_imshow
import cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00


# Run setup

install images from specified tator.csv file. example csv file can be found in github under `tests/test_data`

In [ ]:
!kelp-coverage setup --tator-csv seymour.csv --tator-token <api-token>

# Run Kelp detecion and coverage calculation
if tator-csv doesn't match with given setup tator.csv the program will silently fail. coverage / segmentation masks declared will be created but the results.json file will fail to be created

example image:

In [ ]:
img = cv2.imread('/content/images/trinity-2_20250404T173830/trinity-2_20250404T173830_Seymour_DSC02153.JPG')
new_width = int(img.shape[1] / 2)
new_height = int(img.shape[0] / 2)
new_dim = (new_width, new_height)
resized_img = cv2.resize(img, new_dim, interpolation=cv2.INTER_AREA)
cv2_imshow(resized_img)

The base analyze function will generate a coverage percentage in `results.json` in the run directory and a kelp mask under `masks`. If a `site` is not provided then the program will go over all images within the site that it has not already processed.

In [ ]:
!kelp-coverage analyze \
    --site "trinity-2_20250404T173830" \
    --images /content/images/trinity-2_20250404T173830/trinity-2_20250404T173830_Seymour_DSC02153.JPG \
    --tator-csv seymour.csv

img = cv2.imread('/content/results/trinity-2_20250404T173830/4005f6fd/masks/trinity-2_20250404T173830_Seymour_DSC02153_kelp_mask.png')
new_width = int(img.shape[1] / 2)
new_height = int(img.shape[0] / 2)
new_dim = (new_width, new_height)
resized_img = cv2.resize(img, new_dim, interpolation=cv2.INTER_AREA)
cv2_imshow(resized_img)

The output saved to `results/<site>/<hash>/visualizations/results.json` will look like this with the arguments and commands passed in:
```python
{
    "command": "/usr/local/bin/kelp-coverage analyze \
    --site trinity-2_20250404T173830 \
    --images /content/images/trinity-2_20250404T173830/ trinity-2_20250404T173830_Seymour_DSC02153.JPG \
    --tator-csv seymour2.csv",
    "run_args": {
        "command": "analyze",
        "sam_checkpoint": "mobile_sam.pt",
        "use_mobile_sam": true,
        ...
    },
```
and results for each individual image:
```python
    "results": [
        {
            "image_name": "trinity-2_20250404T173830_Seymour_DSC02153.JPG",
            "image_id": 444550,
            "latitude": 36.94621202698628,
            "longitude": 122.06924938798072,
            "coverage_percentage": 1.6009210428483418
        }
        ...
    ]
```

helpful flags:

`--generate-slice-viz`: generates a slice by slice visualization of the SAM model segmentation process \
`--generate-overlay`: generates an overlay of the final segmentation mask over the original image \
`--generate-component-viz` [HEIRARCHICAL]: generates an overlay of the the two generated masks over the original image \
`--overwrite`: reruns the program over an image even if its calculation has already been saved in `results.json` \
`--help`: lists all available functions \
`--verbose`: verbose output

# Run debug
to better visualize what is going on within a slice run and adjust the parameters for thresholding use the debug command

In [7]:
!kelp-coverage debug-slice \
  --site "trinity-2_20250404T173830" \
  --image-path /content/images/trinity-2_20250404T173830/trinity-2_20250404T173830_Seymour_DSC02153.JPG \
  --visualize-stages \
  --slice-index 5


--- Running Debug on: trinity-2_20250404T173830_Seymour_DSC02153.JPG ---
Saved debug visualization(s) to: results/debug

--- Debug processing complete. Visualizations saved in: results/debug ---


In [ ]:
import matplotlib.pyplot as plt
s1 = cv2.imread('/content/results/debug/trinity-2_20250404T173830_Seymour_DSC02153_slice_5_thresh20_stage1_grid_validation.png')
s2 = cv2.imread('/content/results/debug/trinity-2_20250404T173830_Seymour_DSC02153_slice_5_thresh20_stage2_point_selection.png')
s3 = cv2.imread('/content/results/debug/trinity-2_20250404T173830_Seymour_DSC02153_slice_5_thresh20_stage3_final_selection.png')
s4 = cv2.imread('/content/results/debug/trinity-2_20250404T173830_Seymour_DSC02153_slice_5_thresh20_stage4_final_overlay.png')

images_to_show = [s1, s2, s3, s4]

fig, axes = plt.subplots(1, 4, figsize=(30, 20))
for i, image in enumerate(images_to_show):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    axes[i].imshow(rgb_image)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Generate Heatmap
after running over all images generate a heatmap of the site

In [10]:
!kelp-coverage heatmap --coverage-data "example_res.json"

Generating heatmap...
Heatmap saved to results/heatmap/20250404T173830_heatmap.png


In [ ]:
img = cv2.imread('results/heatmap/20250404T173830_heatmap.png')
new_width = int(img.shape[1] / 2)
new_height = int(img.shape[0] / 2)
new_dim = (new_width, new_height)
resized_img = cv2.resize(img, new_dim, interpolation=cv2.INTER_AREA)
cv2_imshow(resized_img)

# To save the results:

In [ ]:
from google.colab import files
import zipfile
import os
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"results_{timestamp}.zip"
directory_to_zip = "results"

if os.path.exists(directory_to_zip):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files_in_dir in os.walk(directory_to_zip):
            for file in files_in_dir:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, directory_to_zip)
                zipf.write(file_path, arcname)
    print(f"Successfully created {zip_filename}")
    files.download(zip_filename)
else:
    print(f"Error: Directory '{directory_to_zip}' not found.")

Successfully created results_20250807_020337.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>